# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [ ]:
import pandas as pd 
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [ ]:
 #Configurar la aplicación Spark
app_name = "tarea"

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName(app_name).getOrCreate()

### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [ ]:

df = spark.read.csv("/datosTarea.csv", header=True, inferSchema=True)
df.show()
     

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [ ]:
resp = df.filter((df['Name'].startswith('M')) & (df["Number of Employees"].between(4000, 7000))).select("Name", "Number of Employees")

resp.show()

>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [ ]:
# Filtrar los países que no comienzan con 'b', 's' o 'm' y tienen un networth menor o igual a 500000
result_df = df.filter(
    (~col("Country").startswith("b")) &
    (~col("Country").startswith("s")) &
    (~col("Country").startswith("m")) &
    (col("Networth") <= 500000)
)

# Mostrar el resultado
result_df.select("Name", "Country", "Networth").show()

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [ ]:
def subtract_mean(pdf):
    return pdf.assign(profit_minus_mean=pdf['profit'] - pdf['profit'].mean())

### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [ ]:
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

# Agrupar por industry y encontrar las empresas con el profit más alto
result_df = df.groupBy("Industry").agg({"Profit": "max"}).withColumnRenamed("max(Profit)", "max_profit")

# Ordenar por profit de manera descendente y mostrar los tres primeros
result_df = result_df.orderBy(desc("max_profit")).limit(3)

# Mostrar el resultado
result_df.show()

>Agrupa por industry y calcula el promedio de empleados que tienen

In [ ]:
from pyspark.sql.functions import avg
# Agrupar por industry y calcular el promedio de empleados
result_df = df.groupBy("Industry").agg(avg("Number of Employees"))

# Mostrar el resultado
result_df.show()

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [ ]:
df.createOrReplaceTempView("empresa_table")
query = """
    SELECT COUNT(*) as count_empresas
    FROM empresa_table
    WHERE Founded > 2000
"""

result = spark.sql(query)

result.show()

### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [ ]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Ensamblar las características en un vector
assembler = VectorAssembler(
    inputCols=["Number of employees", "Networth", "stock_price"],
    outputCol="features"
)
df_assembled = assembler.transform(df)
# Seleccionar las columnas relevantes
data = df_assembled.select("features", "Profit")

# Dividir el conjunto de datos en entrenamiento y prueba
train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)

# Configurar el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="Profit")

# Entrenar el modelo
lr_model = lr.fit(train_data)

# Realizar predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [ ]:
# Convertir el DataFrame de PySpark a un DataFrame de pandas
pandas_df = predictions.select("features", "profit", "prediction").toPandas()

# Guardar el DataFrame de pandas como un archivo CSV
pandas_df.to_csv("resultados.csv", index=False)